In [114]:
def getVarFromFile(filename):
    import importlib.machinery
    import importlib.util
    loader = importlib.machinery.SourceFileLoader('data', filename)
    spec = importlib.util.spec_from_loader(loader.name, loader)
    global data
    data = importlib.util.module_from_spec(spec)
    loader.exec_module(data)

personal_path = '329047-parameters.txt'
test_path = '123-params.txt'
getVarFromFile(personal_path)

<big><b>EXERCISE 1</b></big>

In [41]:
Q1_N = data.Q1_N
Q1_e = data.Q1_e
Q1_L_mod = data.Q1_L_mod
Q1_L_x = data.Q1_L_x

In [42]:
for Ni in Q1_L_mod:
    p = gcd(Q1_N, Ni)
    if p != 1:
        break

In [46]:
print(is_prime(p))
qr = ZZ(Q1_N / p)

True


In [47]:
for i in range(len(Q1_L_x)):
    if Q1_L_x[i][0]!=Q1_L_x[i][1]:
        indx = i
        break

print(indx)
pt, d_ct = Q1_L_x[indx]

9


In [48]:
mod(pt, p) == mod(d_ct, p)

True

In [49]:
#x = mod(pt, qr)
#x_prime = mod(d_ct, qr)
q = gcd(pt-d_ct, qr)
r = ZZ(qr / q)
print(is_prime(p))
print(is_prime(q))
print(is_prime(r))
print(p==q)
print(p==r)
print(q==r)
print(Q1_N == (p*q*r))

phi_N = (p-1)*(q-1)*(r-1)
d = inverse_mod(Q1_e, phi_N)

True
True
True
False
False
False
True


In [51]:
import hashlib

Q1_ct_RSA = data.Q1_ct_RSA
Q1_ct_pwd = data.Q1_ct_pwd

R = Integers(Q1_N)
a = R(Q1_ct_RSA)
x_key = ZZ(a^d)

key = hashlib.sha512(str(x_key).encode('ascii')).digest()
m = bytes(a ^^ b for (a, b) in zip(key, Q1_ct_pwd))
print(m.decode('ascii'))

pharaoh55


<big><b>EXERCISE 2</b></big>

In [78]:
Q2_k = data.Q2_k
Q2_G = data.Q2_G
Q2_ciphertexts = data.Q2_ciphertexts 

In [79]:
print(Q2_k)
exponents = [e for N,e in Q2_G]
num_of_exp = {e:exponents.count(e) for e in exponents}
import operator
sorted_noe = sorted(num_of_exp.items(), key=operator.itemgetter(1), reverse=True)
print(sorted_noe[:10])

818
[(61, 61), (1319, 4), (1883, 4), (1909, 4), (1669, 4), (1433, 3), (1579, 3), (1793, 3), (1259, 3), (1723, 3)]


In [80]:
small_exp = sorted_noe[0][0]
remainders = []
moduli = []
for i in range(Q2_k):
    if Q2_G[i][1] == small_exp:
        remainders.append(Q2_ciphertexts[i])
        moduli.append(Q2_G[i][0])
m_exp = CRT_list(remainders, moduli)
print(m_exp^(1/small_exp))

705042596582157880382819988


<big><b>EXERCISE 3A</b></big>

In [181]:
Q3a_e = ZZ(data.Q3a_e)
Q3a_N = ZZ(data.Q3a_N)

In [182]:
e = Q3a_e
N = Q3a_N
cf = (e/N).continued_fraction()
for a in cf.convergents():
    h = a.numerator()
    d = a.denominator()
    if(h!=0 and (e * d - 1) % h == 0):
        totient_N = (e * d - 1)/h
        if(2313.powermod(totient_N, N) == 1): #change this hardcoded value
            b = N - totient_N + 1; c = N
            roots = solve([x^2-b*x+c==0],x)
            for q in roots:
                print(gcd(N, q.rhs()))

1032671771770553847758184444937001839020394770768176909480719145263523827277732798293782651420057081508620026872997436757954296094349781306743484926843029810060447419425004800503194998191966295274434066908596422357062670874692763067
1552247184011206077597178405928627479752260679801635882417663485026400339979517492763896486027861576017987790002212618690418878946842041239588989562515497587574658841760267496929513539741730954152425310347975860813627609336556499969


In [184]:
p = 1032671771770553847758184444937001839020394770768176909480719145263523827277732798293782651420057081508620026872997436757954296094349781306743484926843029810060447419425004800503194998191966295274434066908596422357062670874692763067
q = 1552247184011206077597178405928627479752260679801635882417663485026400339979517492763896486027861576017987790002212618690418878946842041239588989562515497587574658841760267496929513539741730954152425310347975860813627609336556499969
print(is_prime(p))
print(is_prime(q))
print(Q3a_N == p*q)

True
True
True


<big><b>EXERCISE 3B</b></big>

In [110]:
Q3b_e = data.Q3b_e
Q3b_N = data.Q3b_N
Q3b_c1 = data.Q3b_c1
Q3b_c2 = data.Q3b_c2

In [21]:
def extended_euclides(a,b,quo=lambda a,b:a//b):
    r0 = a; r1 = b
    s0 = 1; s1 = 0
    t0 = 0; t1 = 1

    while r1 != 0:
        q = quo(r0, r1)
        r0, r1 = r1, r0 - q * r1
        s0, s1 = s1, s0 - q * s1
        t0, t1 = t1, t0 - q * t1

    lc = r0.lc()
    return r0/lc, s0/lc, t0/lc

In [60]:
def common_root(f1, f2):
    gcd_poly, _, _ = extended_euclides(f1, f2)
    common_root = gcd_poly.small_roots()[0]
    return common_root

In [93]:
def find_message(g1, g2, R):
    
    P_x.<x> = PolynomialRing(R, implementation='NTL')
    P_y.<y> = PolynomialRing(R, 1)
    
    h = g1.change_ring(P_y).resultant(g2.change_ring(P_y), x)
    U.<y> = PolynomialRing(R, implementation='NTL')
    h = U(h)
    delta = h.small_roots(X=2^16, beta=0.5)[0]
    
    return common_root(P_x(g1(x, delta)), P_x(g2(x, delta)))

In [112]:
e = Q3b_e
c1 = Q3b_c1
c2 = Q3b_c2

R = Zmod(Q3b_N)
P.<x,y> = PolynomialRing(R, 2)

g1 = x^e - c1
g2 = (x+y)^e - c2
M1 = find_message(g1, g2, R)

g1 = x^e - c2
g2 = (x+y)^e - c1
M2 = find_message(g1, g2, R)

k = 2^floor(RR(log(Q3b_N, 2)) / 9)
r1 = IntegerMod(Zmod(k), M1)
r2 = IntegerMod(Zmod(k), M2)

P2.<x> = PolynomialRing(R, implementation='NTL')
f1 = k*x + P2(r1) - M1
f2 = k*x + P2(r2) - M2
int_M = Integer(common_root(f1, f2))

hex_M = int_M.hex()
M = bytearray.fromhex(hex_M).decode()
print(M)


Life is an incessant series of problems... all difficult, with brutally limited choices... and a time limit. (Chapel the Evergreen, Trigun)
